<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [3]:
!pip install sqlalchemy==1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [4]:
!pip install ipython-sql
!pip install ipython-sql prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [5]:
%load_ext sql

In [6]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [7]:
!pip install -q pandas

In [8]:
%sql sqlite:///my_data1.db

In [9]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [10]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [11]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [23]:
cur.execute('SELECT DISTINCT "LaunchSite" FROM SPACEXTABLE;')

# Fetch the results
launch_sites = cur.fetchall()

# Print the results
for site in launch_sites:
    print(site)

# Close the connection
#conn.close()

('LaunchSite',)



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [25]:
cur.execute('SELECT * FROM SPACEXTABLE WHERE "LaunchSite" LIKE "CCA%" LIMIT 5;')

# Fetch the results
records = cur.fetchall()

# Check if there are any records to print
if records:
    # Print the results
    print("Records where LaunchSite begins with 'CCA':")
    for record in records:
        print(record)
else:
    print("No records found that match the condition.")


No records found that match the condition.


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [26]:
cur.execute('''
    SELECT SUM("PayloadMass") AS total_payload_mass
    FROM SPACEXTABLE
    WHERE "Mission" LIKE '%CRS%' OR "Customer" LIKE '%NASA%';
''')

# Fetch the result
total_payload_mass = cur.fetchone()

# Print the result
if total_payload_mass:
    print(f"Total Payload Mass carried by NASA CRS launches: {total_payload_mass[0]} kg")
else:
    print("No matching records found.")

Total Payload Mass carried by NASA CRS launches: 0.0 kg


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [27]:
cur.execute('''
    SELECT AVG("PayloadMass") AS avg_payload_mass
    FROM SPACEXTABLE
    WHERE "BoosterVersion" = 'F9 v1.1';
''')

# Fetch the result
avg_payload_mass = cur.fetchone()

# Print the result
if avg_payload_mass:
    print(f"Average Payload Mass carried by F9 v1.1: {avg_payload_mass[0]} kg")
else:
    print("No matching records found.")

Average Payload Mass carried by F9 v1.1: None kg


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [28]:
cur.execute('''
    SELECT MIN("Date") AS first_successful_landing_date
    FROM SPACEXTABLE
    WHERE "Landing_Outcome" = 'Success' AND "LandingPad" IS NOT NULL;
''')

# Fetch the result
first_landing_date = cur.fetchone()

# Print the result
if first_landing_date:
    print(f"First successful landing date on ground pad: {first_landing_date[0]}")
else:
    print("No matching records found.")

First successful landing date on ground pad: 2018-07-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [29]:
cur.execute('''
    SELECT DISTINCT "BoosterVersion"
    FROM SPACEXTABLE
    WHERE "Landing_Outcome" = 'Success'
    AND "LandingPad" LIKE '%Drone Ship%' 
    AND "PayloadMass" > 4000 
    AND "PayloadMass" < 6000;
''')

# Fetch the result
boosters = cur.fetchall()

# Print the result
if boosters:
    print("Boosters with successful landing on drone ship and payload mass between 4000 and 6000:")
    for booster in boosters:
        print(booster[0])
else:
    print("No matching boosters found.")


No matching boosters found.


### Task 7




##### List the total number of successful and failure mission outcomes


In [31]:
cur.execute('''
    SELECT "Landing_Outcome", COUNT(*) AS total_outcomes
    FROM SPACEXTABLE
    GROUP BY "Landing_Outcome";
''')

# Fetch the results
outcomes = cur.fetchall()

# Print the result
if outcomes:
    print("Total number of successful and failed mission outcomes:")
    for outcome in outcomes:
        print(f"{outcome[0]}: {outcome[1]}")  # outcome[0] is the Landing_Outcome and outcome[1] is the count
else:
    print("No matching mission outcomes found.")

Total number of successful and failed mission outcomes:
Controlled (ocean): 5
Failure: 3
Failure (drone ship): 5
Failure (parachute): 2
No attempt: 21
No attempt : 1
Precluded (drone ship): 1
Success: 38
Success (drone ship): 14
Success (ground pad): 9
Uncontrolled (ocean): 2


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [32]:
cur.execute('''
    SELECT "BoosterVersion"
    FROM SPACEXTABLE
    WHERE "PayloadMass" = (
        SELECT MAX("PayloadMass") 
        FROM SPACEXTABLE
    );
''')

# Fetch the result
boosters = cur.fetchall()

# Print the result
if boosters:
    print("Booster versions that carried the maximum payload mass:")
    for booster in boosters:
        print(booster[0])
else:
    print("No matching boosters found.")

Booster versions that carried the maximum payload mass:
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion
BoosterVersion

### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [33]:
cur.execute('''
    SELECT 
        CASE 
            WHEN substr("Date", 6, 2) = '01' THEN 'January'
            WHEN substr("Date", 6, 2) = '02' THEN 'February'
            WHEN substr("Date", 6, 2) = '03' THEN 'March'
            WHEN substr("Date", 6, 2) = '04' THEN 'April'
            WHEN substr("Date", 6, 2) = '05' THEN 'May'
            WHEN substr("Date", 6, 2) = '06' THEN 'June'
            WHEN substr("Date", 6, 2) = '07' THEN 'July'
            WHEN substr("Date", 6, 2) = '08' THEN 'August'
            WHEN substr("Date", 6, 2) = '09' THEN 'September'
            WHEN substr("Date", 6, 2) = '10' THEN 'October'
            WHEN substr("Date", 6, 2) = '11' THEN 'November'
            WHEN substr("Date", 6, 2) = '12' THEN 'December'
        END AS Month,
        "Landing_Outcome", 
        "BoosterVersion", 
        "LandingPad", 
        "LaunchSite"
    FROM SPACEXTABLE
    WHERE substr("Date", 0, 5) = '2015'
      AND "Landing_Outcome" = 'Failure'
      AND "LandingPad" LIKE '%Drone Ship%';
''')

# Fetch the results
records = cur.fetchall()

# Print the result
if records:
    print("Failed landing outcomes on drone ships in 2015:")
    for record in records:
        print(f"Month: {record[0]}, Outcome: {record[1]}, Booster: {record[2]}, LandingPad: {record[3]}, LaunchSite: {record[4]}")
else:
    print("No matching records found.")

No matching records found.


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
